In [1]:
from pydub import AudioSegment

In [2]:
%pip install pocketsphinx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 7.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 3.9 MB/s eta 0:00:00
  Created wheel for pocketsphinx: filename=pocketsphinx-5.0.3-cp39-cp39-macosx_10_16_x86_64.whl size=29109730 sha256=ef5b4518800bd497aedee6db377faecd65c28b61bdd8716e31a661ebee9d1da4
  Stored in directory: /Users/joregan/Library/Caches/pip/wheels/63/2e/68/74d27530bb89b696aed5ddf017d2293c90744253d5692b4f36
Successfully built pocketsphinx
Note: you may need to restart the kernel to use updated packages.


In [13]:
MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
"""

In [5]:
EGTEXT = "Yeah, that's true. I mean, they are the same size and they are a little bit, but I think I I should go more for something that style."
EGPHON = "/jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ˈɑːɹ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl./"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_7_0719_210_001_main.wav"
EGSTART = 70.028
EGEND = 75.441

In [11]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [normphon(x) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output
    

In [12]:
make_lexicon(EGTEXT, EGPHON)

[('bit', 'bˈɪɾ'),
 ('but', 'bˈʌt'),
 ('little', 'lˈɪɾə'),
 ('style', 'stˈaɪl'),
 ('more', 'mˈɔːɹ'),
 ('size', 'sˈaɪz'),
 ('are', 'ˈɑːɹ'),
 ('that', 'ðˈæt'),
 ('are', 'ɚ'),
 ('the', 'ðə'),
 ('something', 'sˈʌmθɪŋ'),
 ('they', 'ðˈeɪ'),
 ('should', 'ʃˈʊ'),
 ('true', 'tɹˈuː'),
 ('same', 'sˈeɪm'),
 ('i', 'ˈaɪ'),
 ('i', 'ə'),
 ('and', 'ən'),
 ('go', 'ɡˈoʊ'),
 ('think', 'θˈɪŋk'),
 ('yeah', 'jˈæ'),
 ('for', 'fɚ'),
 ('a', 'ə'),
 ('mean', 'mˈiːn'),
 ("that's", 'ðˈæs')]